In [1]:
import psycopg2
from psycopg2 import sql
import pandas as pd

In [2]:
import psycopg2

def create_database(dbname, user, password, host="localhost", port="5432"):
    """Creates a PostgreSQL database."""
    try:
        # Connect to the PostgreSQL server (not a specific database yet)
        conn = psycopg2.connect(host=host, port=port, user=user, password=password, dbname="postgres")
        conn.autocommit = True  # Enable autocommit for DDL statements like CREATE DATABASE
        cur = conn.cursor()

        # Check if the database already exists
        cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", (dbname,))
        exists = cur.fetchone()

        if not exists:
            # Create the database
            cur.execute(f"CREATE DATABASE {dbname}")
            print(f"Database '{dbname}' created successfully.")
        else:
            print(f"Database '{dbname}' already exists.")

        cur.close()
        conn.close()

    except psycopg2.Error as e:
        print(f"Error creating database: {e}")
    finally:
        # Ensure the connection is closed
        if conn:
            conn.close()

# Example usage (replace with your credentials):
dbname = "school_fee_db"
user = "uthkarsh"
password = "Ruthwik081@"

create_database(dbname, user, password)

# Now you can connect to the created database:
try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host="localhost", port="5432")
    print(f"Connected to database '{dbname}'")
    # ... perform other database operations ...
    conn.close()
except psycopg2.Error as e:
    print(f"Error connecting to database: {e}")


Database 'school_fee_db' already exists.
Connected to database 'school_fee_db'


In [3]:
def create_tables(dbname, user, password):
    try:
        # Connect to the school_fee_db
        conn = psycopg2.connect(dbname=dbname, user=user, password=password, host="localhost", port="5432")
        cursor = conn.cursor()

        # CREATE TABLE IF NOT EXISTSs
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS Transport (
            transport_id SERIAL PRIMARY KEY,
            pick_up_point VARCHAR(100),
            route_number INT
        );

        CREATE TABLE IF NOT EXISTS Student (
            admission_number INT PRIMARY KEY,
            student_name VARCHAR(100),
            guardian_name VARCHAR(100),
            gender VARCHAR(10),
            aadhar_number BIGINT,
            date_of_birth DATE,
            date_of_joining DATE,
            contact_number VARCHAR(15),
            village VARCHAR(100),
        );

        CREATE TABLE IF NOT EXISTS ClassDetails (
            admission_number INT REFERENCES Student(admission_number),
            year INT,
            class INT,
            section VARCHAR(2),
            roll_number INT,
            photo_id INT,
            language VARCHAR(50),
            vocational VARCHAR(50),
            currently_enrolled BOOLEAN,
            PRIMARY KEY (admission_number, year)
        );

        CREATE TABLE IF NOT EXISTS Fee (
            admission_number INT REFERENCES Student(admission_number),
            year INT,
            school_fee DECIMAL(10, 2),
            concession_reason VARCHAR(50),
            
            -- Automatically calculated school_fee_concession based on concession_reason
            school_fee_concession DECIMAL(10, 2) GENERATED ALWAYS AS (
                CASE 
                    WHEN concession_reason = 'Staff' THEN school_fee / 2
                    WHEN concession_reason = 'Sibbling' THEN school_fee / 10
                    WHEN concession_reason = 'OTP' THEN school_fee / 10
                    WHEN concession_reason = 'TF' THEN school_fee / 20
                    WHEN concession_reason = 'FP' THEN school_fee / 5
                    WHEN concession_reason = 'EC' THEN school_fee * 0.15
                    WHEN concession_reason = 'SC' THEN school_fee / 4
                    WHEN concession_reason = '' THEN 0
                    WHEN concession_reason = 'General' THEN school_fee / 10
                    ELSE 0
                END
            ) STORED,
            transport_used BOOLEAN,
            application_fee DECIMAL(10, 2),
            transport_fee DECIMAL(10, 2),
            transport_fee_concession DECIMAL(10, 2),
            transport_id INT REFERENCES Transport(transport_id),
            PRIMARY KEY (admission_number, year)
        );

        CREATE TABLE IF NOT EXISTS FeeBreakdown (
            admission_number INT REFERENCES Student(admission_number),
            year INT,
            fee_type VARCHAR(50),
            term VARCHAR(20),
            paid DECIMAL(10, 2),
            due DECIMAL(10, 2),
            receipt_no INT,
            fee_paid_date DATE,
            PRIMARY KEY (admission_number, year, fee_type)
        );

        CREATE TABLE IF NOT EXISTS YearlyRecord (
            admission_number INT REFERENCES Student(admission_number),
            year INT,
            total_fee_paid DECIMAL(10, 2),
            total_fee_due DECIMAL(10, 2),
            PRIMARY KEY (admission_number, year)
        );
        """)

        conn.commit()
        print("Tables created successfully.")

        cursor.close()
        conn.close()
    except Exception as e:
        print("Error creating tables:", e)


In [4]:
create_tables(dbname, user, password)

Tables created successfully.
